In [6]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
xml_dir = '/data/darknet/python/' + time_str + '_xml_expand_obj/' 
#xml_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_whgk/VOC2007/Annotations/'
image_dir = '/data/darknet/python/' + time_str + '_image_expand_obj/'
#image_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_whgk/VOC2007/JPEGImages/'
generated_image_dir = image_dir
#'/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_GuAn/VOC2007/JPEGImages/'
generated_xml_dir = xml_dir
#'/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_GuAn/VOC2007/Annotations/'

#os.system('mkdir -p ' + xml_dir)
#os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
os.system('mkdir -p ' + generated_xml_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]


def get_fliped_bbox(bbox, img_w):
    return (img_w-bbox[0], img_w-bbox[1], bbox[2], bbox[3])

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
    
def generate_new_xmlobj(xmlobj_old, new_bbox):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text
    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text
    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(int(new_bbox[0]))
    tag_ymin.text = str(int(new_bbox[2]))
    tag_xmax.text = str(int(new_bbox[1]))
    tag_ymax.text = str(int(new_bbox[3]))
    return element_object

def generate_etree(etree_old):
    root_old = etree_old.getroot()    
    root = ET.Element('annotation')
    root.append(root_old.find('folder'))
    root.append(root_old.find('filename'))
    root.append(root_old.find('source'))
    root.append(root_old.find('size'))
    root.append(root_old.find('segmented'))    
    tree = ET.ElementTree(root)    
    return tree

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    

In [7]:
seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()
print(len(seed_xml_names))
for xml_name in seed_xml_names:
        img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
        xml_name = SEED_XML_DIR + xml_name
        print(img_name)
        print(xml_name)
        
        tree=ET.parse(xml_name)
        new_tree = generate_etree(tree)
        img = cv2.imread(img_name)
        print(img.shape)
        
        objs = get_obj_from_xml(xml_name)
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)
            new_bbox = get_fliped_bbox(bbox, img.shape[1])
            #print(bbox)
            #print(new_bbox)
            new_obj_element = generate_new_xmlobj(obj_element, new_bbox)
            new_tree = insert_to_xml(new_obj_element, new_tree)
        time_mark = datetime.datetime.now()
        time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
        new_tree.write(GENE_XML_DIR + 'fliped_' + time_str + xml_name.split('/')[-1])
        h_flip = cv2.flip(img, 1)
        cv2.imwrite(GENE_IMG_DIR + 'fliped_' + time_str + img_name.split('/')[-1], h_flip)

5702
/data/darknet/python/20190425_image_expand_obj/20190425092831389745_00003002089.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425092831389745_00003002089.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425092832476897_00003002090.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425092832476897_00003002090.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425092833335034_00003002091.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425092833335034_00003002091.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425092834271364_00003002092.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425092834271364_00003002092.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425092835241323_00003002093.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425092835241323_00003002093.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425092836269741_00003002094.jpg
/

(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093130970742_00003002352.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093130970742_00003002352.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093131875880_00003002353.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093131875880_00003002353.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093133313121_00003002355.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093133313121_00003002355.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093134220083_00003002356.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093134220083_00003002356.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093135000963_00003002357.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093135000963_00003002357.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093151535227_000030

(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093300865779_00003002513.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093300865779_00003002513.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093301750359_00003002514.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093301750359_00003002514.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093302719584_00003002515.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093302719584_00003002515.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093303531732_00003002516.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093303531732_00003002516.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093304276389_00003002517.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093304276389_00003002517.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093305103359_000030

/data/darknet/python/20190425_image_expand_obj/20190425093351983764_00003002585.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093351983764_00003002585.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093352807346_00003002586.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093352807346_00003002586.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093353625915_00003002587.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093353625915_00003002587.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093355523666_00003002590.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093355523666_00003002590.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093356373533_00003002591.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425093356373533_00003002591.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425093357221211_00003002592.jpg
/data/

(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425100407827023_00003006014.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425100407827023_00003006014.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425100408570699_00003006015.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425100408570699_00003006015.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425100410184580_00003006018.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425100410184580_00003006018.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425100410907415_00003006019.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425100410907415_00003006019.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425100412306046_00003006021.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425100412306046_00003006021.xml
(1920, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425100413034821_000030

...

/data/darknet/python/20190425_image_expand_obj/20190425163949490516_00003006154.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425163949490516_00003006154.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425163949984806_00003006155.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425163949984806_00003006155.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425163950882816_00003006156.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425163950882816_00003006156.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425163951468821_00003006157.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425163951468821_00003006157.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425163952037453_00003006158.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425163952037453_00003006158.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425163952653630_00003006159.jpg
/data/

/data/darknet/python/20190425_image_expand_obj/20190425164013770981_00003006204.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164013770981_00003006204.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164014125477_00003006205.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164014125477_00003006205.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164014481706_00003007000.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164014481706_00003007000.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164014820641_00003007001.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164014820641_00003007001.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164015181696_00003007002.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164015181696_00003007002.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164015535287_00003007003.jpg
/data/

(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164036745623_00003007063.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164036745623_00003007063.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164037305794_00003007064.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164037305794_00003007064.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164037785234_00003007065.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164037785234_00003007065.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164038339886_00003007066.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164038339886_00003007066.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164038691362_00003007067.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164038691362_00003007067.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164039221602_000030

(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164102555833_00003007113.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164102555833_00003007113.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164103166639_00003007114.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164103166639_00003007114.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164103927402_00003007115.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164103927402_00003007115.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164104771904_00003007116.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164104771904_00003007116.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164105367255_00003007117.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164105367255_00003007117.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164105861117_000030

/data/darknet/python/20190425_image_expand_obj/20190425164141269981_00003007163.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164141269981_00003007163.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164141839123_00003007164.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164141839123_00003007164.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164142551029_00003007165.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164142551029_00003007165.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164142945328_00003007166.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164142945328_00003007166.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164143494884_00003007168.jpg
/data/darknet/python/20190425_xml_expand_obj/20190425164143494884_00003007168.xml
(1370, 1080, 3)
/data/darknet/python/20190425_image_expand_obj/20190425164144044672_00003007169.jpg
/data/